# UCERF3 using the ```loaducerf3``` module
This notebook loads all of the UCERF3 data using the ```loaducerf3``` python module. It is not required for a general REHEATFUNQ analysis, and is not suitable for regions other than California. Furthermore, due to the history of REHEATFUNQ, the notebook is more powerful than needed for a REHEATFUNQ analysis. The only part of this notebook that is needed for a REHEATFUNQ analysis is the fault trace. Since the `loaducerf3` module handling of UCERF3 logic tree branches is quite computationally heavy, the `REHEATFUNQ_ONLY` switch is provided in this notebook:

In [ ]:
REHEATFUNQ_ONLY = False

To perform the full UCERF3 logic tree branch analysis, set it to **`False`**. If you want to apply REHEATFUNQ to another area, proceed to section [Fault Trace](#Fault-Trace) and adjust the code according to your data.

Configure plots to look good on a HiDPI monitor (you may not need the following configuration if you are not using a HiDPI monitor):

In [ ]:
%config InlineBackend.figure_format = 'retina'

General imports:

In [ ]:
import requests
import numpy as np
from pathlib import Path
from zipfile import ZipFile
from flottekarte import Map
from itertools import product
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
from pickle import Pickler, Unpickler
from matplotlib.patches import Polygon as MPolygon
from loaducerf3 import UCERF3Model, UCERF3Files, Polygon, PolygonSelector

In [ ]:
from zeal2022hf import UCERF3BranchResult

## UCERF3 Branching Structure
The following code sets up the loading of the UCERF3 files from the "Full Model (Compound Solutions)" of [Milner (2014)](https://doi.org/10.5281/zenodo.5519801).

#### Download
First, this code can download and extract the `full_ucerf3_compound_sol.zip`, `branch_averaged_ucerf3_sol_FM3_1.zip`, and `relm_gridded_region.csv` files from Milner (2014).
Alternatively, the contents of the archives and files can be copied to the `UCERF3_DATA_PATH` specified below, or
a suitable file system link can be set.

In [ ]:
UCERF3_DATA_PATH = 'data/full_compound_solution/'

In [ ]:
def load_ucerf3_compound_sol(download = False):
    """
    This function downloads the UCERF3 'full_ucerf3_compound_sol.zip'
    file and extracts the data.
    """
    file = Path('data/full_ucerf3_compound_sol.zip')
    file.parent.mkdir(exist_ok=True)
    
    # Download the file if it does not exist:
    if not file.is_file() and download:
        link = "https://zenodo.org/record/5519802/files/full_ucerf3_compound_sol.zip?download=1"
        r = requests.get(link)
        with open(file,'wb') as f:
            f.write(r.content)
    
    # Unzip the file into the UCERF3_DATA_PATH:
    data_path = Path(UCERF3_DATA_PATH)
    if not data_path.is_dir() or not any(data_path.iterdir()):
        with ZipFile(file, 'r') as f:
            f.extractall(data_path)


def load_ucerf3_grid_sources_bin(download=False):
    """
    This function extracts the 'grid_sources.bin' file from
    the 'branch_averaged_ucerf3_sol_FM3_1.zip' file (and downloads
    this archive if needed).
    """
    file = Path('data/branch_averaged_ucerf3_sol_FM3_1.zip')
    file.parent.mkdir(exist_ok=True)
    
    
    # Download the file if it does not exist:
    if not file.is_file() and download:
        link = "https://zenodo.org/record/5519802/files/branch_averaged_ucerf3_sol_FM3_1.zip?download=1"
        r = requests.get(link)
        with open(file,'wb') as f:
            f.write(r.content)
    
    # Unzip the file into the UCERF3_DATA_PATH:
    grid_sources_bin_path = Path(UCERF3_DATA_PATH) / "grid_sources.bin"
    if not grid_sources_bin_path.is_file():
        with ZipFile(file, 'r') as f:
            f.extract('grid_sources.bin', grid_sources_bin_path.parent)

def download_ucerf3_relm_gridded_region_csv():
    """
    This function ensures that the `relm_gridded_region.csv` file exists in
    the data directory, downloading it if not.
    """
    file = Path(UCERF3_DATA_PATH) / 'relm_gridded_region.csv'
    file.parent.mkdir(exist_ok=True)

    if not file.is_file():
        link = "https://zenodo.org/record/5519802/files/relm_gridded_region.csv?download=1"
        r = requests.get(link)
        with open(file,'wb') as f:
            f.write(r.content)

This following code downloads the `full_ucerf3_compound_sol.zip` from [zenodo.org](https://zenodo.org/record/5519802).

**Note:** The download of the archives is disabled by default so as not to cause unnecessary traffic.
If you have already downloaded a copy of `full_ucerf3_compound_sol.zip` and/or `branch_averaged_ucerf3_sol_FM3_1.zip`, please copy the file(s) into the `data/` directory (or whichever
other parent directory you have chosen for `UCERF3_DATA_PATH` above). If you wish to download using this
notebook, change the below lines to

```python
load_ucerf3_compound_sol(download=True)
load_ucerf3_grid_sources_bin(download=True)
```


In [ ]:
load_ucerf3_compound_sol()
load_ucerf3_grid_sources_bin()
download_ucerf3_relm_gridded_region_csv()

#### Access the extracted file system structure
The following code iterates the extracted files. The file indexing is done according to the
branches, which iterate the fault model, deformation model, earthquake scaling relation by
rupture length, slip distribution, $M>5$ event rate, $M_\mathrm{max}$ choice, and off-fault
smoothed seismicity version (Field *et al.*, 2014).

We index the branches by a vector of 7 integers $\vec{i}=(i_0, i_1, i_2, i_3, i_4, i_5, i_6)$.
Each of the integers indexes one of the logic tree branches mentioned before.

| Index | Dimension                  | Index value | Model parameter  |
| :---: | :------------------------: | :---------: | :--------------: |
| $i_0$ | Fault model                |      0      |     FM3.1        |
|       |                            |      1      |     FM3.2        |
| $i_1$ | Deformation model          |      0      |      ABM         |
|       |                            |      1      |      GEOL        |
|       |                            |      2      |      NEOK        |
|       |                            |      3      |     ZENGBB       |
| $i_2$ | EQ scaling from rup. len.  |      0      |      EllB        |
|       |                            |      1      |   EllBsqrtLen    |
|       |                            |      2      |      HB08        |
|       |                            |      3      |    Shaw09Mod     |
|       |                            |      4      |   ShConStrDrp    |
| $i_3$ | Slip Distribution          |      0      |     DsrTap       |
|       | Slip Distribution          |      1      |     DsrUni       |
| $i_4$ | $M>5$ event rate           |      0      |       6.5        |
|       |                            |      1      |       7.9        |
|       |                            |      2      |       9.6        |
| $i_5$ | $M_\mathrm{max}$ off-fault |      0      |       7.3        |
|       |                            |      1      |       7.6        |
|       |                            |      2      |       7.9        |
| $i_6$ | Off-fault seism. version   |      0      |    SpatSeisU2    |
|       |                            |      1      |    SpatSeisU3    |


For description on the model parameters, we refer to Field *et al.* (2014). The branches are weighted
as proposed by them.

The following functions iterate the branch structure.
 - `file_name` returns the file name for each one of the relevant files (`grid_sources.bin`, `rup_sections.bin`, 
   `rup_lengths.bin`, `fault_sections.xml`, `sect_areas.bin`, `rup_areas.bin`, `mags.bin`, and `rates.bin`)
   for each of the UCERF3 branches.
 - `branches` is an iterator over all branches of the UCERF3 model, yielding a tuple
   `(UCERF3Files, float, UCERF3Model)` (where the middle entry is the branches weight) for each of the branches.

In [ ]:
# All the logic tree branching points.
# We save a logic tree branch as a tuple (level, name, #branches, options, weights)
# Weights are taken from Field et al. (2014), BSSA
BRANCHES = ((0,'fault-model',       2, ('FM3_1','FM3_2'),                       (0.5, 0.5)),
            (1,'deformation-model', 4, ('ABM','GEOL','NEOK','ZENGBB'),          (0.1, 0.3, 0.3, 0.3)),
            (2,'eq-scale-relation', 5, ('EllB','EllBsqrtLen','HB08','Shaw09Mod','ShConStrDrp'), 
                                                                                (0.2, 0.2, 0.2, 0.2, 0.2)),
            (3,'slip-distribution', 2, ('DsrTap','DsrUni'),                     (0.5, 0.5)),
            (4,'M>5-event-rate',    3, ('M5Rate6.5','M5Rate7.9','M5Rate9.6'),   (0.1, 0.6, 0.3)),
            (5,'Mmax-off-fault',    3, ('MMaxOff7.3','MMaxOff7.6','MMaxOff7.9'),(0.1, 0.8, 0.1)),
            (6,'off-fault-version', 2, ('SpatSeisU2','SpatSeisU3'),             (0.5, 0.5))
           )

def file_name(branch, ftype='rates.bin'):
    """
    Implement the UCERF3 naming scheme:
    """
    S = UCERF3_DATA_PATH
    # There is only one grid_sources.bin
    if ftype == 'grid_sources.bin':
        return S + 'grid_sources.bin'
    
    # Rupture sections change only with fault model:
    S += BRANCHES[0][3][branch[0]]
    if ftype == 'rup_sections.bin':
        return S + '_rup_sections.bin'
    if ftype == 'rup_lengths.bin':
        return S + '_rup_lengths.bin'
    
    # Fault sections and section areas change additionally
    # with deformation model:
    S += '_' + BRANCHES[1][3][branch[1]]
    
    if ftype == 'fault_sections.xml':
        return S + '_fault_sections.xml'
    if ftype == 'sect_areas.bin':
        return S + '_sect_areas.bin'
    if ftype == 'rup_areas.bin':
        return S + '_rup_areas.bin'
    
    S += '_' + BRANCHES[2][3][branch[2]]
    
    # Magnitudes change only for the first three logic tree branch levels:
    if ftype == 'mags.bin':
        return S + '_mags.bin'

    # Rates change with all levels:
    if ftype == 'rates.bin':
        return S + '_' + BRANCHES[3][3][branch[3]] + '_CharConst_' + BRANCHES[4][3][branch[4]] \
                 + '_' + BRANCHES[5][3][branch[5]] + '_NoFix_' + BRANCHES[6][3][branch[6]] + '_rates.bin'

def segment_selector(fault_segment, roi_id):
    return np.all(selectors[roi_id].mask(fault_segment))

In [ ]:
def branches(selector, proj_str, select_if='any'):
    """
    Iterates the branches and yields the models.
    """
    for b0 in range(BRANCHES[0][2]):
        for b1 in range(BRANCHES[1][2]):
            for b2 in range(BRANCHES[2][2]):
                for b3 in range(BRANCHES[3][2]):
                    for b4 in range(BRANCHES[4][2]):
                        for b5 in range(BRANCHES[5][2]):
                            for b6 in range(BRANCHES[6][2]):
                                branch = (b0,b1,b2,b3,b4,b5,b6)
                                
                                # Branches keywords:
                                if b3 == 0:
                                    slip_model = 'tapered'
                                else:
                                    slip_model = 'uniform'

                                # Filenames:
                                fault_sections_xml = file_name(branch, 'fault_sections.xml')
                                mags_bin = file_name(branch, 'mags.bin')
                                rates_bin = file_name(branch, 'rates.bin')
                                rup_areas_bin = file_name(branch, 'rup_areas.bin')
                                rup_lengths_bin = file_name(branch, 'rup_lengths.bin')
                                rup_sections_bin = file_name(branch, 'rup_sections.bin')
                                sect_areas_bin = file_name(branch, 'sect_areas.bin')
                                grid_sources_bin = file_name(branch,'grid_sources.bin')
                                relm_gridded_region_csv = UCERF3_DATA_PATH + 'relm_gridded_region.csv'
                                
                                files = UCERF3Files(fault_sections_xml, mags_bin, rates_bin,
                                                    rup_areas_bin, rup_lengths_bin, rup_sections_bin,
                                                    sect_areas_bin, grid_sources_bin,
                                                    relm_gridded_region_csv)

                                model = UCERF3Model(files, proj_str, selector, select_if,
                                                    use_slip=True, use_area=True,
                                                    consider_intersegment_distance=True,
                                                    consider_masked_segments=True,
                                                    drop_incomplete_ruptures=False,
                                                    slip_weighting=slip_model)


                                weight = BRANCHES[0][4][b0] * BRANCHES[1][4][b1] \
                                         * BRANCHES[2][4][b2] * BRANCHES[3][4][b3] \
                                         * BRANCHES[4][4][b4] * BRANCHES[5][4][b5] \
                                         * BRANCHES[6][4][b6]

                                yield branch, weight, model

## Process the branch structure
The following function processes one branch of the UCERF3 model, given a region to analyize,
and returns the relevant results:
 - branch weight
 - fraction of radiated power in the region which is radiated power on the (San Andreas) fault
 - radiated power on the (San Andreas) fault
 - total radiated power of the branch model within the region
 - total power radiated from faults in the region (total radiated power minus the off-fault power)
 - the sum of the products $L_i d_i^2 v_i$ with $i$ iterating the fault segments of the (San Andreas)
   fault within the region. $L$ is the segment's length, $d$ its depth, and $v$ its slip rate. This
   product will be important to compute the frictional power using the friction coefficient.
 - the sum of $d_i^2$ for all (San Andreas) fault segments
 - the sum of slip rates $v_i$ of all (San Andreas) fault segments.

In [ ]:
def compute_xi_ld2v_for_ucerf3_branches(general_poly, proj_str, power_poly=None, power_select_mode='any'):
    # Iterate over the branches:
    res = {}
    res["elements"] = ("weight", "saf_relative_power", "saf_power",
                       "total_power", "fault_power", "ld2v", "d2", "sr")
    
    assert power_select_mode in ('any','all')
    
    selector = PolygonSelector(Polygon(*general_poly.T))
    if power_poly is not None:
        power_select_one = PolygonSelector(Polygon(*power_poly.T))
    
    def _power_segment_selector(seg):
        """
        A fault segment selector for computing the power of a subset
        of the fault segments.
        """
        # First make sure that we select only segments of the San Andreas:
        if "San Andreas" not in seg.parent_name():
            return False
        
        # Then make sure that we are conforming to the (optional)
        # further selector:
        if power_poly is not None:
            if power_select_mode == 'any':
                return np.any(power_select_one.array_mask(seg.coordinates()))
            return np.all(power_select_one.array_mask(seg.coordinates()))

        return True


    for branch, weight, model in branches(selector, proj_str):
        #
        # Power on the San Andreas fault:
        #
        saf_power = sum(fs.power() for fs in model.fault_segments(select=_power_segment_selector))

        # For comparison: Total power only on faults.
        fault_power = sum(f.power() for f in model.faults())

        # Relative power with regards to the sum of fault power and grid source power:
        saf_relative_power = saf_power / model.total_power()

        # Compute the total velocity-squared depth product
        # (this will be needed to compute the frictional power):
        ld2v = sum(s.length() * (s.area() / s.length())**2 * s.slip_rate()
                   for s in model.fault_segments(select=_power_segment_selector))

        # Compute the total d_i^2:
        d2 = sum((s.area() / s.length())**2
                 for s in model.fault_segments(select=_power_segment_selector))

        # Compute the total slip rate:
        sr = sum(s.slip_rate() for s in model.fault_segments(select=_power_segment_selector))

        res[branch] = UCERF3BranchResult(weight, saf_relative_power, saf_power, model.total_power(),
                                         fault_power, ld2v, d2, sr)
        
        # Some diagnostic output to see the progress of this fairly long computation:
        print("branch:",branch)
        print("   relative:",saf_power / model.total_power(),"\n")
                                
    return res

#### Defining the regions

First, define the two types of regions as defined by the selection polygons:
 1. Regions which contain the San Andreas fault (all)
 2. Regions which are inside the ENCOS area (0,1)

In [ ]:
with open('intermediate/02-Geometry.pickle','rb') as f:
    geometry = Unpickler(f).load()

In [ ]:
proj_str = geometry["proj_str"]
ENCOS_poly_xy = geometry["encos_poly_xy"]
selection_polys_xy = geometry["selection_polygons_xy"]

In [ ]:
def limits(xy):
    xlim = np.min(xy[:,0]), np.max(xy[:,0])
    ylim = np.min(xy[:,1]), np.max(xy[:,1])
    return xlim, ylim

In [ ]:
fig = plt.figure(); ax = fig.add_subplot(111)
mp = Map(proj_str, ax, *limits(np.concatenate([ENCOS_poly_xy] + selection_polys_xy, axis=0)))
ax.add_patch(MPolygon(ENCOS_poly_xy, facecolor='none', edgecolor='tab:blue'))
for i,poly in enumerate(selection_polys_xy):
    ax.add_patch(MPolygon(poly, facecolor='none', edgecolor='tab:orange'))
    ax.text(*poly.mean(axis=0), str(i))
mp.plot_axes()

In [ ]:
REGIONS_WITH_SAF = [0,1,2,3]
REGIONS_WITH_SAF_AND_ENCOS = [0,1]

#### Heavy lifting
Now perform the numerical analyis of the UCERF3 branches. Since this step can take
quite a while, we use toggleable caching.

In [ ]:
USECACHE = True

In [ ]:
if not REHEATFUNQ_ONLY:
    if USECACHE and Path('data/05-UCERF3-branches.pickle').is_file():
        with open('data/05-UCERF3-branches.pickle','rb') as f:
            UCERF3_branch_results = Unpickler(f).load()
    else:
        # The regions 
        UCERF3_branch_results = []
        for i in REGIONS_WITH_SAF:
            UCERF3_branch_results.append(
                compute_xi_ld2v_for_ucerf3_branches(selection_polys_xy[i][:-1,:], proj_str, None,
                                                    power_select_mode='any')
            )

        # Finally append the analysis comparing the power to the whole ENCOS ROI:
        for i in REGIONS_WITH_SAF:
            UCERF3_branch_results.append(
                compute_xi_ld2v_for_ucerf3_branches(ENCOS_poly_xy, proj_str, selection_polys_xy[i][:-1,:],
                                                    power_select_mode='any')
            )

        # Caching:
        with open('data/05-UCERF3-branches.pickle','wb') as f:
            Pickler(f).dump(UCERF3_branch_results)

## Fault Trace
Here, we load the fault trace(s) from our default branch of the UCERF3 model. To apply the
REHEATFUNQ workflow to another region of interest, the fault trace would have to be loaded
by other means. The trick here would be to recreate the following two variables:
 - `surface_traces`: A list `[x0, ..., xi, ..., xn]` of `n` 2D numpy arrays of shape $(m_i,2)$
   (type `list[np.ndarray]`), where $m_i$ is the number of points in fault trace $i$ and the
   second axis enumerates $(x,y)$ coordinates.
 - `merged_surface_traces`: A combined version of the individual traces. The function `merge_traces`
   can be used for it.
  
The following `merge_traces` function is an ad-hoc implementation of a fault trace merger. It tries to connect an unsorted set of surface fault trace segments into a continous fault trace.

In [ ]:
def merge_traces(traces):
    """
    Merges surface traces.
    
    The algorithm is not super robust - it assumes that endpoints
    of following traces are close together. In particular, closer
    than the traces are long.
    """
    # A list
    #
    # [((x10,y10),(x11,y11)), ((x20,y20),(x21,y21)), ..., ((xn0,yn0),(xn1,yn1))]
    #
    # that contains, for each i in [1,...,n] the entpoints of trace[i]
    endpoints = [t[(0,-1), :] for t in traces]
    
    # The same, flattened, so that trace[i] maps to elements endpoints_flat[2*i], endpoints_flat[2*i+1]:
    endpoints_flat = np.concatenate(endpoints,axis=0)

    # Mapping end points to nodes:
    tree = KDTree(endpoints_flat)
    
    # Start with an extremal point:
    xmin,xmax = float(endpoints_flat[:,0].min()), float(endpoints_flat[:,0].max())
    ymin,ymax = float(endpoints_flat[:,1].min()), float(endpoints_flat[:,1].max())
    eplist = [(float(ep[0]), float(ep[1])) for ep in endpoints_flat]
    epset = set(eplist)
    if (xmin,ymin) in epset:
        istart = eplist.index((xmin,ymin))
    elif (xmin,ymax) in epset:
        istart = eplist.index((xmin,ymax))
    elif (xmax,ymin) in epset:
        istart = eplist.index((xmax,ymin))
    elif (xmax,ymax) in epset:
        istart = eplist.index((xmax,ymax))
    
    # The index of the trace:
    i = istart
    t = i // 2
    merged = []
    Nt = len(traces)
    mask = np.zeros(Nt,dtype=bool)
    for j in range(Nt):
        # Trace index:
        if mask[t]:
            #from matplotlib.collection import LineCollection
            fig = plt.figure()
            ax = fig.add_subplot(111)
            if (i % 2) == 0:
                ax.scatter(*traces[t][0])
            else:
                ax.scatter(*traces[t][-1])
            for k in range(Nt):
                ax.plot(*traces[k].T)
            raise RuntimeError()
        mask[t] = True
        if (i % 2) == 0:
            merged.append(traces[t])
            xyend = traces[t][-1,:]
        else:
            merged.append(traces[t][::-1])
            xyend = traces[t][0,:]

        # Next endpoint:
        inext = tree.query(xyend,2)[1]
        if (inext[0] // 2) == t:
            i = int(inext[1])
        else:
            i = int(inext[0])
        
        # Next trace:
        t = i // 2
        if mask[t]:
            # If this failed, there might be something funky with
            # the fault traces. Iterate over the unmasked traces and
            # choose the one with smallest distance to jump to:
            dmin = np.inf
            tmin = 0
            for k in range(Nt):
                if mask[k]:
                    continue
                dmin_k = np.linalg.norm(xyend[np.newaxis,:] - traces[k][(0,-1),:], axis=1).min()
                if dmin_k < dmin:
                    dmin = dmin_k
                    tmin = k
            t = tmin
        
        
    return np.concatenate(merged, axis=0)

In [ ]:
def default_models(selection_polys, default_branch, select_if='any'):
    """
    This function load the default branches.
    """

    UCERF3_models = []
    relm_gridded_region_csv = UCERF3_DATA_PATH + 'relm_gridded_region.csv'
    selectors = []
    for i,poly in enumerate(selection_polys):
        print("Reading model",i+1,"of",len(selection_polys))
        # The selector for the current polygon:
        selector = PolygonSelector(Polygon(*poly[:-1,:].T))
        selectors.append(selector)

        # Files:
        fault_sections_xml = file_name(default_branch, 'fault_sections.xml')
        mags_bin = file_name(default_branch, 'mags.bin')
        rates_bin = file_name(default_branch, 'rates.bin')
        rup_areas_bin = file_name(default_branch, 'rup_areas.bin')
        rup_lengths_bin = file_name(default_branch, 'rup_lengths.bin')
        rup_sections_bin = file_name(default_branch, 'rup_sections.bin')
        sect_areas_bin = file_name(default_branch, 'sect_areas.bin')
        grid_sources_bin = file_name(default_branch,'grid_sources.bin')
    
        #branch = (b0,b1,b2,b3,b4,b5,b6)

        # Branches keywords:
        if default_branch[4] == 0:
            slip_model = 'tapered'
        else:
            slip_model = 'uniform'

        files = UCERF3Files(fault_sections_xml, mags_bin, rates_bin,
                        rup_areas_bin, rup_lengths_bin, rup_sections_bin,
                        sect_areas_bin, grid_sources_bin,
                        relm_gridded_region_csv)

        model = UCERF3Model(files, proj_str, selector, select_if,
                            use_slip=True, use_area=True,
                            consider_intersegment_distance=True,
                            consider_masked_segments=True,
                            drop_incomplete_ruptures=False,
                            slip_weighting=slip_model)

        UCERF3_models.append(model)


    def saf_selector(seg):
        return "San Andreas" in seg.parent_name()
        
    # Lengths of the faults and area slip rate:
    d = []
    ld2v = []
    Av = []
    dA = []
    for i,model in enumerate(UCERF3_models):
        print("i=",i)
        L = sum(s.length() for s in model.fault_segments(select=saf_selector))
        A = sum(s.area() for s in model.fault_segments(select=saf_selector))
        
        # Average depth:
        print("A=",A)
        print("L=",L)
        d.append(A / L)

        # Integral of d^2*v along the fault trace (required for computing the total force on the fault plane):
        ld2v.append(sum(s.length() * (s.area() / s.length())**2 * s.slip_rate()
                        for s in model.fault_segments(select=saf_selector)))
    
    # Surface traces:
    surface_traces = []
    for i,model in enumerate(UCERF3_models):
        trace_i = []
        for fault_seg in model.fault_segments(select=saf_selector):
            trace_i.append(fault_seg.coordinates())

        surface_traces.append(merge_traces(trace_i))
    
    return d, ld2v, surface_traces

In [ ]:
default_branch = (0,3,3,0,1,1,1)
d, ld2v, surface_traces = default_models(selection_polys_xy, default_branch)

In [ ]:
merged_surface_trace = merge_traces(surface_traces)

Inspect the merge fault trace:

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
for i in range(4):
    ax.scatter(*surface_traces[i].T)
    ax.scatter(*surface_traces[i][(0,-1),:].T, marker='o', s=100, facecolor='none',
               edgecolor='k')
ax.plot(*merged_surface_trace.T, color='k', linewidth=0.9);

### Combined Map
Here, show the geometry of `02-Study-Area-Geometry` and the fault trace cropped to those regions:

In [ ]:
fig = plt.figure(); ax = fig.add_subplot(111)
mp = Map(proj_str, ax, *limits(np.concatenate([ENCOS_poly_xy] + selection_polys_xy, axis=0)))
ax.add_patch(MPolygon(ENCOS_poly_xy, facecolor='none', edgecolor='tab:blue'))
ax.plot(*merged_surface_trace.T, color='gray', linewidth=0.9)
for i,poly in enumerate(selection_polys_xy):
    ax.add_patch(MPolygon(poly, facecolor='none', edgecolor='tab:orange'))
    ax.text(*poly.mean(axis=0), str(i))
mp.plot_axes()

### Export Data
Finally, here we save the data computed before for loading to other notebooks.
When applying REHEATFUNQ to another study area, only the key `"surface_traces"`
is important and needs to be adjusted to contain the regions' fault traces.

In [ ]:
with open('intermediate/05-SoCal-UCERF3-default-branches.pickle','wb') as f:
    Pickler(f).dump({"d" : d, "ld2v" : ld2v, "surface_traces" : surface_traces,
                     "merged_surface_trace" : merged_surface_trace})

### References
>  Milner, Kevin R. (2014). Third Uniform California Earthquake Rupture Forecast (UCERF3) Fault System Solutions (1.0) [Data set].
>      Zenodo. https://doi.org/10.5281/zenodo.5519802
>
> Edward H. Field, Ramon J. Arrowsmith, Glenn P. Biasi, Peter Bird, Timothy E. Dawson, Karen R. Felzer,
>     David D. Jackson, Kaj M. Johnson, Thomas H. Jordan, Christopher Madden, Andrew J. Michael,
>     Kevin R. Milner, Morgan T. Page, Tom Parsons, Peter M. Powers, Bruce E. Shaw, Wayne R. Thatcher,
>     Ray J. Weldon, Yuehua Zeng; Uniform California Earthquake Rupture Forecast, Version 3
>     (UCERF3)—The Time‐Independent Model. Bulletin of the Seismological Society of America 2014;
>     104 (3): 1122–1180. doi: https://doi.org/10.1785/0120130164

### License
```
A notebook to compute radiated energy, fault geometry, and slip
rates from the UCERF3 model branches.

This file is part of the REHEATFUNQ model.

Author: Malte J. Ziebarth (ziebarth@gfz-potsdam.de)

Copyright © 2019-2022 Deutsches GeoForschungsZentrum Potsdam,
            2022 Malte J. Ziebarth
            

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.
```